In [79]:
from faker import Faker
import random
import pandas as pd
import numpy as np
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from base64 import b64encode, b64decode

# Initialize faker
fake = Faker()

class AESCipher:
    def __init__(self, key):
        self.key = key

    def encrypt(self, data):
        cipher = AES.new(self.key, AES.MODE_EAX)
        ciphertext, tag = cipher.encrypt_and_digest(data)
        return cipher.nonce, b64encode(ciphertext).decode('utf-8')

# Define a function to generate a customer
def generate_customer_profile():
    profile = fake.profile()
    return {
        "name": profile['name'],
        "address": fake.address().replace('\n', ', '),
        "email": profile['mail'],
        "phone_number": fake.phone_number(),
        "social_security_number": fake.ssn(),
        "job": profile['job'],
        "company": profile['company'],
        "residence": fake.address().replace('\n', ', '),
        "current_location": fake.address().replace('\n', ', '),
        "website": profile['website'][0] if profile['website'] else None,
        "username": profile['username'],
        "age": fake.random_int(min=18, max=80, step=1),
        "gender" : profile['sex'],
        "Grades": fake.random_int(min=0, max=100, step=1),
        "preferred_language": fake.language_code(),
        "preferred_currency": fake.currency_code(),
        "job_title": profile['job'],
        "national_id": fake.random_int(min=1000000000, max=9999999999, step=1),
        "height": fake.random_int(min=100, max=200, step=1),
        "weight": fake.random_int(min=100, max=200, step=1),
    }

# Define a function to generate a transaction
def generate_transaction(name):
    return {
        "name": name,
        "amount": f"${fake.random_int(min=1, max=1000, step=1):.2f}",
        "date": fake.date_time_between(start_date='-1y', end_date='now', tzinfo=None),
    }

# Generate customers
customers = [generate_customer_profile() for _ in range(10)]
customers_df = pd.DataFrame(customers)



In [80]:

# Generate transactions for each customer randomly picking a customer by name
transactions = [generate_transaction(random.choice(customers)["name"]) for _ in range(20)]
transactions_df = pd.DataFrame(transactions)
transactions_df.head()

,name,amount,date
0,Anthony Gallegos,$834.00,2022-07-06 17:51:49
1,Anthony Gallegos,$351.00,2022-11-08 21:51:03
2,Derek Butler,$887.00,2023-03-08 18:01:10
3,Madison Branch DDS,$852.00,2023-05-16 09:44:55
4,Anthony Gallegos,$529.00,2022-10-30 11:05:55


In [81]:

# Encrypt the data in the CRM
password = get_random_bytes(16)  # In a real system, never hard-code the password
cipher = AESCipher(password)


In [82]:

encrypted_df = pd.DataFrame()
encryption_key = pd.DataFrame()
for column in customers_df.columns:
    encrypted_column = []
    key_column = []
    for data in customers_df[column]:
        iv, ct = cipher.encrypt(str(data).encode('utf-8'))
        encrypted_column.append(ct)
        key_column.append(iv)
    encrypted_df[column] = encrypted_column
    encryption_key[column] = key_column

print("password: ", password)
# save the encrypted data (first five rows as a markdown table)
md_table = encrypted_df.head().to_markdown()
# save md table to file
with open("encrypted_data.md", "w") as f:
    f.write(md_table)

# Read the README.md file
readme_text = ""
with open("README.md", "r") as f:
    readme_text = f.read()
# open the README.md file and add the encrypted data at the end
with open("README.md", "a") as f:
    # read what is currently in the README.md file
    readme = readme_text
    # add the encrypted data
    readme += "\n\n## Encrypted Data\n\n"
    readme += md_table
    # write the new README.md file
    f.write(readme)

# Print the encrypted data
encrypted_df.head()

password:  b'\x8eo\xbb\xe2c\xb4\xc8\xce\x9b\x9f9\x04\xd9\x18\xed\xeb'


,name,address,email,phone_number,social_security_number,job,company,residence,current_location,website,username,age,gender,Grades,preferred_language,preferred_currency,job_title,national_id,height,weight
0,G/JS5hWBTpA0YzrX,VfwfAlvqCjixoR7F45Q2lexZttD3xepeEx5jAaoEfF0tHk...,uXmxzo6r9mzbO//1SFYXVyfL,oRc43zFyErvfknHp,tE0Z7H6JL0I+7eY=,jVnzfVlf6EjoHJcKkIqKSDx+P7wOK7O3akskycHNuU2XjW...,uMk1QQ8Uezy8gAQTbIWrLN5uQe7qJg==,Y6rOhrSfpvU6/bERh4EqYpt8d+1G8NXjaKMs9w2O1KXe,MZzAm6YzCTDPDYKbJnbtS7FaxK6nCgRbiBDgUBTBCIPnB0...,X4REvrA6LFJRKTLtm3nA1IbcbQ==,9LLiNxFzOh9xiIKE,tyE=,Aw==,ttg=,e3I=,+bfu,KWIlwNtAx+hiJ8IIbsQDlOX7weRWIZxdjBjBWog4MDjfbC...,Q+Nh6kWUilWVPQ==,8r67,R4+q
1,kGVjKKOBIJ22oyybekpVpTJa,qDsWJRK4yLSIH0YrjCTcslRUv/WCUkkNp4OljjSmNunyW3...,kMf6MDl67CtHxB4sNz3uOBXUr5dK,mLAEWECtFVm/+Jzg7tefiw==,uf0pK+9qlhE8/X4=,ZPMvWrxFZNC3DKZe0rJiDDD0Ag==,k4SKtwwp0Zd+jXM=,OX5GcRk8EDogNEXWNcW6RvIA7HFW81Ryt3t9/r2QOrtNPb...,p0owcI90mC9+MJqDwrV4MuXMhg6vkjmR/JNjA4rV1TmGTl...,psEqPsjVK3L7bSQlvVNl42OBAqcICn8JZ9c=,GUUWOiWgzQ==,MYs=,mg==,rA==,XXE=,z+LR,3P0iF+3WgT0bsmo3RSkdJ2fZww==,B2NZVKm62l9cxw==,5c0E,WJ6F
2,7+k8BwWS7Ej9//NU1V4y5Q==,ixzyOgWXhpObgX1pcCY/cOhcQqy00T+tcSZPy0tvrfywY0...,VDslIaMTYXyzYS5zl5Ptt2JDnTwtLfQ=,jLY8RH81kksqb+qJhpjB+HP8HOKNmg==,+aYdrJkrQ8nvP74=,vr5L4gkowDnnrRX245UJLmpV8hidY4LXVyAE,9cF8CKy7U/H6gHBjqXo=,9ei7EEHwLFooXqStP2GYQdqnvbNpZIbUWYIRo8Ym7R0Mht...,DTqnsrTgcNQir81eNFzSPGg1m7il07213Fg/A6FU7IMZGi...,G+NYEYWybIF+s209Tku7pr2CLfrqUHAQYw==,LGj78D3OKw==,bes=,SA==,fbk=,aLc=,jMPo,ND0IeMs7pBGmC5WT7/P//FJl6A/TyDfv7Pz+,fi5UCStWaLtl8w==,bIAH,O+eR
3,jSsNrgUnpnCADHQMGRx4mg==,dXxMG0LwscExRJoqJ4+ZJjOx1JDR44jc4vxI6bZVgDoadP...,Wg27KyK5gKYkdAsEoM7/D3E=,BDAWStWLryH3Jr9qULSYUQ==,sdMedHWnsVqrD20=,fVnZtdBOOyDKZ58J+h+HZ8DqWs9AVk7daA15cmY=,Dh+h8hM0I+rV2CW9aChyRL6GV3+2VAQ=,rKacemZvDrwjSccAu8WkJsDOUg+I56AU7Nb/OHE8WONFXA...,BZYD63027+RiPgCNiO/Xp86ouKZveXCIMnmzzKVABA2ANe...,GpxwUc3Y2MvV2iOuAvYB1AuK/TGk,l9L5oW01cw==,7V8=,Kw==,cJo=,dsQ=,jzDP,SGpBSYzdtYs2eU/p+O37Y/Nj5uyYAvBc8RnnoE8=,xyirhnfq6aA89Q==,VAmT,f7vr
4,CRBRB3MBH4l9bbpkxJ4=,W4aWCuqzUCKfh2SS2e0f6phBedeYq3sz6api8iPCUSQPAv...,wODqkqgB5e8HE65YXG7h3JiDSA==,M7NhZgdNbkxJTZ7oOcfV+v0=,ZlPE8BcPUCEVe5s=,qOd4TXnqtK5ZP73uFjHWZA2ozQ==,gKLN4GwDjrwFJA2X,4370vHMclQF7xy7061+6gKeM9A/8zw7fbuEGZLLQh4RqhM...,QKsI1PY7vxz4pqUzW5ARBaOgLIV/zYG6PUtsv5jmZsurl2nG,ZaDiFaiAdNXrr1F0azi1xf0rbM8xjsAekehqdg==,Iq+pZFwbfhI=,G0M=,dg==,pw==,QuU=,Fu9q,IOcMVc4hAuJGqqmKbt7qikkRkQ==,de6n8H5+GGzj0Q==,p5un,CD0f
